In [38]:
# import libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import MapboxTiles
from cartopy.io.img_tiles import GoogleWTS
from owslib.wmts import WebMapTileService
from owslib.wms import WebMapService
from urllib.request import urlopen
from PIL import Image
import geopandas as gpd
import os
import io
from dotenv import load_dotenv
load_dotenv() # loads .env file from main folder for Mapbox API key and local project path
import random
import numpy as np

In [4]:
# load parcel and building data
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Spatial Data/ny-manhattan-parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
df_buildings = gpd.read_file(os.environ.get('LOCAL_PATH')+"Spatial Data/ny-manhattan-buildings/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [5]:
# set random RGB color for the parcels
def random_hex_color(seed=False):
  if seed:
    random.seed(seed)
    r = random.randint(0, 255)
    random.seed(seed+1000)
    g = random.randint(0, 255)
    random.seed(seed+2000)
    b = random.randint(0, 255)
  else:
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
  return "#{:02x}{:02x}{:02x}".format(r, g, b)
random_hex_color()
df_parcels['color'] = [ random_hex_color() for i in range(len(df_parcels)) ]

In [6]:
# join data frames function
def join_parcels_buildings(parcels, buildings):
    """Join parcels and buildings dataframes."""
    parcels_buildings = buildings.sjoin(parcels, how="inner")
    return parcels_buildings

In [7]:
# set CRS to web mercator
df_parcels = df_parcels.to_crs(epsg=3857)
df_buildings = df_buildings.to_crs(epsg=3857)

In [8]:
# perform join
df_parcels_buildings = join_parcels_buildings(df_parcels, df_buildings)

In [9]:
# show df length
print(df_parcels.shape[0])
print(df_buildings.shape[0])
print(df_parcels_buildings.shape[0])

42297
1084092
94448


In [10]:
# loop through the parcels and add them to the map
def add_geometries(ax, df_parcels, crs_epsg, random_color = False):
    for row in df_parcels.itertuples():
        geometry = row.geometry
        if random_color == True: color = random_hex_color(int(row.bin))
        else: color = row.color
        ax.add_geometries(geometry, crs = crs_epsg, facecolor=color) # for Lat/Lon data.

In [59]:
# Option 1: Mapbox satellite imagery

# load the background map and plot the geometries
# def map_maker(df_parcels, df_buildings, scale=10):
def map_maker(df_parcels, df_buildings, bounds, index, scale=10, feature_type='both', random_color=False):
    access_token = os.environ.get('MAPBOX_ACCESS_TOKEN')
    tiler = MapboxTiles(access_token, 'satellite-v9')
    crs_epsg = ccrs.epsg('3857')

    mercator = tiler.crs

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection=mercator)

    # change figure size of the subplot
    my_dpi=96
    fig.set_size_inches(7, 7)
    # fig.figsize = (512/my_dpi, 512/my_dpi), dpi=my_dpi

    # calculate the centroid and max distance of the bounds
    dist1 = bounds[2]-bounds[0]
    dist2 = bounds[3]-bounds[1]
    max_dist = max(dist1, dist2)/2

    # calculate the centroid of the bounds
    centroid_x = (bounds[2]+bounds[0])/2
    centroid_y = (bounds[3]+bounds[1])/2

    # bounds = df_parcels.total_bounds with offset to create same aspect ratio
    ax.set_extent([centroid_x-max_dist, centroid_x+max_dist, centroid_y-max_dist, centroid_y+max_dist], crs=ccrs.epsg('3857'))

    # if feature_type == 'parcels': add_parcels(ax, df_parcels, crs_epsg)
    if feature_type == 'parcels': 
        add_geometries(ax, df_parcels, crs_epsg)
        # ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='white', edgecolor='black', linewidth=2.5, alpha=1)
    if feature_type == 'parcels' and random_color == True:
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if feature_type == 'buildings': 
        add_geometries(ax, df_buildings, crs_epsg)
    if feature_type == 'buildings' and random_color == True:
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
    if feature_type == 'both' and random_color == True: 
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if  feature_type == 'both': 
        # ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='black', edgecolor='white', linewidth=1.5, alpha=1)
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg)

    # add the Mapbox tiles to the axis at zoom level 10 (Mapbox has 23 zoom levels)
    ax.add_image(tiler, scale)

    # set the path to the folder where the images will be saved
    output_folder = 'img-mapbox/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # save the figure
    plt.savefig(output_folder + f'{feature_type}_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)
    # plt.savefig(output_folder + f'building_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)

    # close the figure
    plt.close(fig)

    # ax.coastlines('10m')
    plt.show()

In [57]:
# Option 2: NASA GIBS (REST) API

def map_maker(df_parcels, df_buildings, bounds, index, scale=10, feature_type='both', random_color=False):
    # Coordinate Reference System for the map
    crs_epsg = ccrs.epsg('3857')
    
    # Construct the tile URL using GIBS API for a single tile as an example
    layer = "MODIS_Terra_CorrectedReflectance_TrueColor"
    date = "2020-03-01"  # Example date, adjust as necessary
    zoom_level = 6  # Zoom level
    tile_row = 10  # Tile row
    tile_col = 21  # Tile column
    tile_url = f"https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/{layer}/default/{date}/GoogleMapsCompatible_Level9/{zoom_level}/{tile_row}/{tile_col}.jpg"
    
    # Setup CartoPy with Mercator projection
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
    ax.set_extent([-130, -100, 20, 50], crs=ccrs.PlateCarree())  # Adjust these bounds

    # calculate the centroid and max distance of the bounds
    dist1 = bounds[2]-bounds[0]
    dist2 = bounds[3]-bounds[1]
    max_dist = max(dist1, dist2)/2

    # calculate the centroid of the bounds
    centroid_x = (bounds[2]+bounds[0])/2
    centroid_y = (bounds[3]+bounds[1])/2

    # bounds = df_parcels.total_bounds with offset to create same aspect ratio
    ax.set_extent([centroid_x-max_dist, centroid_x+max_dist, centroid_y-max_dist, centroid_y+max_dist], crs=ccrs.epsg('3857'))

    # Fetch and display the tile
    with urlopen(tile_url) as url:
        img = Image.open(url)
        img_array = np.array(img)
    img_extent = [-130, -100, 20, 50]  # Adjust this extent to match the tile's coverage
    ax.imshow(img_array, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
    
    # add geometries
    if feature_type == 'parcels': 
        add_geometries(ax, df_parcels, crs_epsg)
    if feature_type == 'parcels' and random_color == True:
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if feature_type == 'buildings': 
        add_geometries(ax, df_buildings, crs_epsg)
    if feature_type == 'buildings' and random_color == True:
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
    if feature_type == 'both' and random_color == True: 
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if  feature_type == 'both': 
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg)
    
    # Output settings
    output_folder = 'img-gibs/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(output_folder + f'{feature_type}_{index}.jpg', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [63]:
def map_maker(df_parcels, df_buildings, bounds, index, scale=10, feature_type='both', random_color=False):
        
    # Connect to GIBS WMS Service
    wms = WebMapService('https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?', version='1.1.1')

    # Configure request for MODIS_Terra_CorrectedReflectance_TrueColor
    img = wms.getmap(layers=['MODIS_Terra_CorrectedReflectance_TrueColor'],  # Layers
                    srs='epsg:4326',  # Map projection
                    bbox=(-180,-90,180,90),  # Bounds
                    size=(1200, 600),  # Image size
                    time='2024-01-01',  # Time of data
                    format='image/jpeg',  # Image format
                    transparent=False)  # Nodata transparency

    # Save output PNG to a file
    output_folder = 'img-gibs/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    out = open(output_folder + f'{feature_type}_{index}.jpg', 'wb')
    out.write(img.read())
    out.close()

In [45]:
# subset the data frames based on a buffer
def subset(df, df_buildings, index, distance = 75):
    selected_feature = df.loc[index]
    geometry_buffer = selected_feature.geometry.buffer(distance)
    geometry_bounds = selected_feature.geometry.buffer(distance-70)

    return df[df.within(geometry_buffer)], df_buildings[df_buildings.within(geometry_buffer)], geometry_bounds.bounds

In [64]:
# generate the image for "parcels" and "buildings" (or "both")
# index will correspond to each of the parcel indices in the data frame
for i in range (0, 10):
    subset_features = subset(df_parcels, df_parcels_buildings, i, 200)
    map_maker(subset_features[0], subset_features[1], subset_features[2], i, 18,'parcels')
    # map_maker(subset_features[0], subset_features[1], subset_features[2], i, 18,'buildings')